In [129]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import os
from pyspark.sql import functions as F

DATA_DIR_M = "/home/masa/Downloads/google-cloud-sdk/our_data"
# DATA_DIR_L =  

In [76]:
# We use spark session in order to use DataFrames
ss = SparkSession.builder \
    .appName("GoogleClusterAnalysis") \
    .master("local[2]") \
    .getOrCreate()

In [77]:
ss.sparkContext.setLogLevel("ERROR")

### 1. What is the distribution of the machines according to their CPU capacity? Can you explain (motivate) it?

In [78]:
machine_schema = StructType([
    StructField('time', LongType(), True),
    StructField('machine_ID', LongType(), True),
    StructField('event_type', IntegerType(), True),
    StructField('platform_ID', StringType(), False),
    StructField('CPUs', DoubleType(), False),
    StructField('memory_capacity', DoubleType(), False),
    ])

In [79]:
df_machines = spark.read.csv(os.path.join(DATA_DIR,"machine_events/*.csv.gz"), schema=machine_schema)
df_machines.show(5)

+----+----------+----------+--------------------+----+---------------+
|time|machine_ID|event_type|         platform_ID|CPUs|memory_capacity|
+----+----------+----------+--------------------+----+---------------+
|   0|         5|         0|HofLGzk1Or/8Ildj2...| 0.5|         0.2493|
|   0|         6|         0|HofLGzk1Or/8Ildj2...| 0.5|         0.2493|
|   0|         7|         0|HofLGzk1Or/8Ildj2...| 0.5|         0.2493|
|   0|        10|         0|HofLGzk1Or/8Ildj2...| 0.5|         0.2493|
|   0|        13|         0|HofLGzk1Or/8Ildj2...| 0.5|         0.2493|
+----+----------+----------+--------------------+----+---------------+
only showing top 5 rows


In [80]:
df_machines.select('machine_ID').distinct().count()

12583

In [81]:
df_machines.select("event_type").distinct().count()

3

In [88]:
df_machines.filter(df_machines["event_type"] == 0).distinct().count()

21443

In [110]:
total_count_t0 = df_machines.filter(df_machines["time"] == 0).distinct().count()
total_count_t0

12477

In [111]:
df_machines.select("machine_ID").filter(df_machines["time"] == 0).distinct().count()

12477

In [112]:
df_machines.select("CPUs").filter(df_machines["time"] == 0).distinct().show()

+----+
|CPUs|
+----+
| 1.0|
| 0.5|
|0.25|
+----+



In [124]:
df_ex1 = df_machines.select(['machine_ID', 'CPUs']).filter(df_machines["time"] == 0).groupby('CPUs').count()
df_ex1.show()

+----+-----+
|CPUs|count|
+----+-----+
| 1.0|  791|
| 0.5|11563|
|0.25|  123|
+----+-----+



In [128]:
df_ex1.withColumn("percentage", F.round((F.col("count") / total_count) * 100)).show()

+----+-----+----------+
|CPUs|count|percentage|
+----+-----+----------+
| 1.0|  791|       6.0|
| 0.5|11563|      93.0|
|0.25|  123|       1.0|
+----+-----+----------+



- at timestamp==0, there are 12477 machines intially started, and for them we want to  find the distribution (of those machines) acoording to their CPU capacity